## 1. PLSC-ViT模型简介


PLSC-ViT实现了基于Transformer的视觉分类模型。ViT对图像进行切分成patch，之后基于patch拉平的sequence进行线性embedding，并且添加了position embeddings和classfication token，然后将patch序列输入到标准的transformer编码器，最终经过一个MLP进行分类。模型结构如下，

![Figure 1 from paper](https://github.com/google-research/vision_transformer/raw/main/vit_figure.png)


## 2. 模型效果 

| Model | Phase | Dataset | gpu | img/sec | Top1 Acc | Official |
| --- | --- | --- | --- | --- | --- | --- |
| ViT-B_16_224 |pretrain  |ImageNet2012  |A100*N1C8  |  3583| 0.75196 | 0.7479 |
| ViT-B_16_384 |finetune  | ImageNet2012 | A100*N1C8 | 719 | 0.77972 | 0.7791 |
| ViT-L_16_224 | pretrain | ImageNet21K | A100*N4C32 | 5256 | - | - |  |
|ViT-L_16_384  |finetune  | ImageNet2012 | A100*N4C32 | 934 | 0.85030 | 0.8505 |

## 3. 模型如何使用

### 3.1 安装PLSC

```
git clone https://github.com/PaddlePaddle/PLSC.git
cd /path/to/PLSC/
# [optional] pip install -r requirements.txt
python setup.py develop
```

### 3.2 模型训练

1. 进入任务目录

```
cd task/classification/vit
```

2. 准备数据

将数据整理成以下格式：
```text
dataset/
└── ILSVRC2012
    ├── train
    ├── val
    ├── train_list.txt
    └── val_list.txt
```

3. 执行训练命令

```shell
export PADDLE_NNODES=1
export PADDLE_MASTER="127.0.0.1:12538"
export CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7

python -m paddle.distributed.launch \
    --nnodes=$PADDLE_NNODES \
    --master=$PADDLE_MASTER \
    --devices=$CUDA_VISIBLE_DEVICES \
    plsc-train \
    -c ./configs/ViT_base_patch16_224_in1k_1n8c_dp_fp16o2.yaml
```

更多模型的训练教程可参考文档：[ViT训练文档](https://github.com/PaddlePaddle/PLSC/blob/master/task/classification/vit/README.md)

### 3.3 模型推理

1. 下载预训练模型

```shell
mkdir -p pretrained/vit/ViT_base_patch16_224/
wget -O ./pretrained/vit/ViT_base_patch16_224/imagenet2012-ViT-B_16-224.pdparams https://plsc.bj.bcebos.com/models/vit/v2.4/imagenet2012-ViT-B_16-224.pdparams
```

2. 导出推理模型

```shell
plsc-export -c ./configs/ViT_base_patch16_224_in1k_1n8c_dp_fp16o2.yaml -o Global.pretrained_model=./pretrained/vit/ViT_base_patch16_224/imagenet2012-ViT-B_16-224 -o Model.data_format=NCHW -o FP16.level=O0
```


## 4. 相关论文及引用信息


```text
@article{dosovitskiy2020,
  title={An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale},
  author={Dosovitskiy, Alexey and Beyer, Lucas and Kolesnikov, Alexander and Weissenborn, Dirk and Zhai, Xiaohua and Unterthiner, Thomas and  Dehghani, Mostafa and Minderer, Matthias and Heigold, Georg and Gelly, Sylvain and Uszkoreit, Jakob and Houlsby, Neil},
  journal={arXiv preprint arXiv:2010.11929},
  year={2020}
}
```